In [9]:
import re
import pandas as pd
from openpyxl import load_workbook
pd.options.display.max_rows = None

In [67]:
# 開啟檔案
def open_brd_file(filepath):
    f = open(filepath)
    return f

# parse net name
def parse_net_name(string):
    # 定義正規表達式
    netNameRex = re.compile(r"^\s-?[A-Z]+[0-9]*_?[A-Z]+[0-9]*.*")
    netName = netNameRex.findall(string)
    return netName

# 找到總長度的欄位
def total(string):
    # 定義正規表達式
    totalRex = re.compile("TOTAL")
    total = totalRex.findall(string)
    return total
#找到位置
def parse_net(string):
#     locationRex = re.compile(r"([U|R|C|L|J]\d+.[A-Z]*\d*|VIA\(T\)|VIA\s)")
    locationRex = re.compile(r"([U|R|C|L|J][A-Z]+\d+\.[A-Z]*\d*|[U|R|C|L|J][A-Z]+\.[A-Z]*\d*|[U|R|C|L|J]\d+.[A-Z]*\d*|[U|R|C|L|J][A-Z]*\d+.[A-Z]*\d*|VIA\(T\)|VIA\s)")
    location = locationRex.findall(string)
    if location and location[0] == "VIA ":
        location[0] = location[0].replace(" ", "")
    return location

#找到單一段長度
def sub_length(string):
    lengthRex = re.compile(r"(\d+.\d+)")
    length = lengthRex.findall(string)[-1]
    return length 

# 找到層數
def layer_num(string):
    layerRex = re.compile(r"BOTTOM|TOP|L\d+$")
    layer = layerRex.findall(string)
    return layer

#找到總長度
def total_length(string):
    ttlLength = re.findall(r"\d+.\d+", string)[0]
    return ttlLength

#存csv
def save_csv(datalist, filename):
    df = pd.DataFrame(datalist)
    df.to_csv(f'{filename}.csv', index = False)
    
# 存excel
def save_excel(write, df, sheetName):
#     df = pd.DataFrame(datalist)
    df.to_excel(write, sheet_name=f'{sheetName}', index=False)
    
#刪多餘的raw
def deleteNullVIAROW(df):
    for i in range(df.shape[0]):
        if df.loc[i, "location"] == "VIA" and df.loc[i, "gap"] == 0.0:
            df.drop(i, axis=0, inplace=True)
            
#這一份txt total的path 數量
def branchPathNum(df):
    #扣掉net name/start_end/total length
    maxPathNum = int((df.shape[1] - 3) / 2)
    return maxPathNum

In [68]:
#step1: 開啟檔案
filepath = r"test2_172.txt"
f = open_brd_file(filepath)
print(f"step1: {filepath} open.")

#step2: 解析檔案
SQS = []
summary = []
netNameList = []

for i in f:
    i = i.replace("\n","")#去掉換行符號
    #如果不是net name 也不是 total
    if not parse_net_name(i):
        if not parse_net(i):
            if not total(i):
                continue
            
    #如果是net name
    if parse_net_name(i):
        data = {} #清空
        netName = i
        #建立netnamelist
        netNameList.append(netName)
        netPath = ""
#         data.update({"net_name" : netName})
#         SQS.append(data)
#         print(data)
    
    #如果是location
    elif parse_net(i):
        data = {} #清空
        
        location = parse_net(i)[0]

        length = float(sub_length(i))
        layer = layer_num(i)[0] if layer_num(i) else ""
        data.update({"net_name" : netName, "location" : location, "length": length, "layer": layer})
#         print(data)
        SQS.append(data)
        
    
    #如果是total那一行
    elif total(i):
        data = {} #清空
        ttlLength = float(total_length(i))
        data.update({"net_name" : netName, "total_length" : ttlLength})
#         print(data)
        summary.append(data)
        
print(f"step2: {filepath} parsed done.")


#存取df
dfsummary, dfSQS, dfSQSR = pd.DataFrame(summary), pd.DataFrame(SQS), pd.DataFrame(SQS)

#修改df
for i in netNameList:
    indexList = dfSQSR[dfSQSR["net_name"] == f"{i}"].index.tolist()
    
    # 填第一個元件的layer空格 
    # ex. TOP = TOP
    dfSQSR.loc[indexList[0],"layer"] = dfSQSR.loc[indexList[1],"layer"]
    
    #每個netname裡面找gap
    length = len(indexList)
    for index in indexList[::-1]:
        if length > 1:
            gap = dfSQSR.loc[index]["length"] - dfSQSR.loc[index-1]["length"]
            dfSQSR.loc[index,"gap"] = gap
        else:
            gap = 0
            dfSQSR.loc[index,"gap"] = gap
        length -= 1
#         print(index, gap)

#刪除多餘的VIA
deleteNullVIAROW(dfSQSR)    
    
#在summary建立branch path & branch length
for i in netNameList:
    indexList = dfSQSR[dfSQSR["net_name"] == f"{i}"].index.tolist()
    path = ""
    connIdxList = []
    
    for idx in indexList:
        if not re.findall("VIA", dfSQSR.loc[idx, "location"]): #找出connnector index 排出VIA, VIA(T)
            connIdxList.append(idx)

    length = len(connIdxList)
    
    #找出connector 在每條indexlist 裡面的index
    num = 0
    netIndex = dfsummary[dfsummary["net_name"] == f"{i}"].index.tolist()
    
    #把起始點&終點 加入summary
    start_end = f"{dfSQSR.loc[indexList[0], 'location']}:{dfSQSR.loc[indexList[-1], 'location']}"

    dfsummary.loc[netIndex, "start_end_path"] = start_end
    
    
    for idx in range(length - 1):
        num +=1 
        pathIdx = []
        pathIdx.append(indexList.index(connIdxList[idx]))
        pathIdx.append(indexList.index(connIdxList[idx + 1]))
               
        #抓出兩兩conn
        connector = ""
        for j in pathIdx:
            connector += dfSQSR.loc[indexList[j], "location"] + ":"
        
        connector = connector[:-1] 
        dfsummary.loc[netIndex, f"path{num}"] = connector
        
        #算出兩兩conn間的長度
        branchLen = 0
        for k in indexList[ pathIdx[0]+1 : pathIdx[1]+1]:
            branchLen += dfSQSR.loc[k, "gap"]

        dfsummary.loc[netIndex, f"length{num}"] = branchLen

        
#調整dfsummary的順序
startEndColumn = dfsummary.pop(dfsummary.columns[2])
dfsummary.insert(1, startEndColumn.name, startEndColumn)
   

#轉成final SQS資料
column1 = []
column2 = []
for row in range(dfsummary.shape[0]):
    column1.append(dfsummary.loc[row,"net_name"])
    column1.append(dfsummary.loc[row,"start_end_path"])
    column2.append("")
    column2.append(dfsummary.loc[row,"total_length"])


    for num in range(branchPathNum(dfsummary)):

        column1.append(dfsummary.loc[row,f"path{num + 1}"])
        column2.append(dfsummary.loc[row,f"length{num + 1}"])
        
final = {
    "SQS"    : column1,
    "length" :column2
}
#轉成df 順便去掉空值
dfFinal = pd.DataFrame(final).dropna(axis=0)        
    
    
# step3: 儲存檔案
filename = re.findall(r"^\w*", filepath)[0]
write = pd.ExcelWriter(f'{filename}.xlsx')
save_excel(write, dfFinal, "final")
save_excel(write, dfsummary, "summary")
save_excel(write, dfSQS, "data base")
save_excel(write, dfSQSR, "SQS")
write.save()

print(f"step3: {filename}.xlsx saved.")

step1: test2_166.txt open.
step2: test2_166.txt parsed done.
step3: test2_166.xlsx saved.


In [62]:
#step1: 開啟檔案
filepath = r"test2_166.txt"
f = open_brd_file(filepath)
print(f"step1: {filepath} open.")

#step2: 解析檔案
SQS = []
summary = []
netNameList = []

for i in f:
    i = i.replace("\n","")#去掉換行符號
    #如果不是net name 也不是 total
    if not parse_net_name(i):
        if not parse_net(i):
            if not total(i):
                continue
            
    #如果是net name
    if parse_net_name(i):
        data = {} #清空
        netName = i
        #建立netnamelist
        netNameList.append(netName)
        netPath = ""
#         data.update({"net_name" : netName})
#         SQS.append(data)
#         print(data)
    
    #如果是location
    elif parse_net(i):
        data = {} #清空
        
        location = parse_net(i)[0]

        length = float(sub_length(i))
        layer = layer_num(i)[0] if layer_num(i) else ""
        data.update({"net_name" : netName, "location" : location, "length": length, "layer": layer})
#         print(data)
        SQS.append(data)
        
    
    #如果是total那一行
    elif total(i):
        data = {} #清空
        ttlLength = float(total_length(i))
        data.update({"net_name" : netName, "total_length" : ttlLength})
#         print(data)
        summary.append(data)
        
print(f"step2: {filepath} parsed done.")


#存取df
dfsummary1, dfSQS1, dfSQSR1 = pd.DataFrame(summary), pd.DataFrame(SQS), pd.DataFrame(SQS)


step1: test2_166.txt open.
step2: test2_166.txt parsed done.


In [63]:
dfSQS1

,net_name,location,length,layer
0,-CLKREQ_PCIE3,UCPU1.DV30,0.000,
1,-CLKREQ_PCIE3,VIA,599.337,BOTTOM
2,-CLKREQ_PCIE3,VIA,599.337,TOP
3,-CLKREQ_PCIE3,VIA,768.006,L8
4,-CLKREQ_PCIE3,VIA,768.006,TOP
5,-CLKREQ_PCIE3,VIA(T),1859.664,BOTTOM
6,-CLKREQ_PCIE3,R7203.1,1896.078,TOP
7,-CLKREQ_PCIE3,U7201.30,1936.255,BOTTOM
8,-CLKREQ_PCIE4,R7001.1,0.000,
9,-CLKREQ_PCIE4,VIA,22.902,TOP


In [30]:
def parse_net_test(string):
    locationRex = re.compile(r"([U|R|C|L|J]\d+.[A-Z]*\d*|[U|R|C|L|J][A-Z]*\d+.[A-Z]*\d*|VIA\(T\)|VIA\s)")
    location = locationRex.findall(string)
    if location and location[0] == "VIA ":
        location[0] = location[0].replace(" ", "")
    return location

In [66]:
#step1: 開啟檔案
filepath = r"test2_166.txt"
f = open_brd_file(filepath)
print(f"step1: {filepath} open.")
for i in f:
#     if parse_net_test(i):
#     locationRex = re.compile(r"([U|R|C|L|J]\d+.[A-Z]*\d*|[U|R|C|L|J][A-Z]*\d+.[A-Z]*\d*|VIA\(T\)|VIA\s)")
#     locationRex = re.compile(r"([U|R|C|L|J]\d+.[A-Z]*\d*|[U|R|C|L|J][A-Z]+.[A-Z]*\d*)")
    locationRex = re.compile(r"([U|R|C|L|J][A-Z]+\d+\.[A-Z]*\d*|[U|R|C|L|J][A-Z]+\.[A-Z]*\d*|[U|R|C|L|J]\d+.[A-Z]*\d*|[U|R|C|L|J][A-Z]*\d+.[A-Z]*\d*|VIA\(T\)|VIA\s)")
    location = locationRex.findall(i)
    print(location)

step1: test2_166.txt open.
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['UCPU1.DV30']
['VIA ']
['VIA ']
['VIA ']
['VIA ']
['VIA(T)']
['R7203.1']
['U7201.30']
[]
[]
[]
['R7001.1']
['VIA ']
['JSSD.52']
['VIA ']
['VIA ']
['UCPU1.DW30']
[]
[]
['CIE3_CLK']
['R7209.1']
['VIA ']
['VIA ']
['VIA ']
['VIA ']
['UCPU1.BY3']
[]
[]
['CIE3_CLK']
['U7201.33']
['R7209.2']
[]
[]
['CIE4_CLK']
['JSSD.53']
['VIA ']
['VIA ']
['VIA ']
['VIA ']
['UCPU1.CN8']
[]
[]
[]
['R1506.2']
['VIA(T)']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['UCPU1.DK39']
['VIA ']
['VIA ']
['VIA ']
['R7218.1']
['VIA ']
['R7002.2']
[]
[]
['CIE3_CLK']
['R7210.1']
['VIA ']
['VIA ']
['VIA ']
['VIA ']
['UCPU1.BY4']
[]
[]
['CIE3_CLK']
['U7201.34']
['R7210.2']
[]
[]
['CIE3_WLAN']
['R7212.1']
['VIA ']
['VIA ']
['VIA ']
['VIA ']
['UCPU1.CT1']
[]
[]
['CIE3_WLAN']
['U7201.36']
['R7212.2']
[]
[]
['CIE3_WLAN']
['R7214.1']
['VIA ']
['VIA ']
['VIA ']
['VIA ']
['UCPU1.CT2']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['CIE3_WLAN']
['U7201.37']
['R7214.2']
[]
[]
['CIE3_WLAN']
['C721

In [16]:
#step1: 開啟檔案
filepath = r"test2_166.txt"
f = open_brd_file(filepath)
print(f"step1: {filepath} open.")

#step2: 解析檔案
SQS = []
summary = []
netNameList = []
for i in f:
    if parse_net_name(i):
            data = {} #清空
            netName = i
            print(i)
            data = {} #清空
            netName = i
            #建立netnamelist
            netNameList.append(netName)
            netPath = ""
    #         data.update({"net_name" : netName})
    #         SQS.append(data)
    #         print(data))
            #建立netnamelist
            netNameList.append(netName)
            netPath = ""
    #         data.update({"net_name" : netName})
    #         SQS.append(data)
    #         print(data)
    
    elif parse_net(i):
        print(i)

step1: test2_166.txt open.
 -CLKREQ_PCIE3

                   UCPU1.DV30*     7309.039 2091.449 L    0.000  

                   VIA             7786.000 2080.130 V  599.337  BOTTOM

                   VIA             7786.000 2080.130 V  599.337  TOP

                   VIA             7872.910 2208.400 V  768.006  L8

                   VIA             7872.910 2208.400 V  768.006  TOP

                   VIA(T)          8207.000 2919.000 V 1859.664  BOTTOM

                   R7203.1*        8171.000 2920.000 L 1896.078  TOP

                   U7201.30*       8173.225 2903.543 L 1936.255  BOTTOM

 -CLKREQ_PCIE4

                   R7001.1*        3838.500  364.300 L    0.000  

                   VIA             3815.628  364.372 V   22.902  TOP

                   VIA             3736.600  131.900 V  317.723  BOTTOM

                   VIA             7351.000 2080.000 V 8144.589  L6

                   UCPU1.DW30*     7334.629 2091.449 L 8165.702  BOTTOM

 -PCIE3_CLK_100M

      

In [10]:
dfSQSR

,net_name,location,length,layer,gap
0,-CLKREQ_PCIE3,U1.DV30,0.000,BOTTOM,0.000
1,-CLKREQ_PCIE3,VIA,599.337,BOTTOM,599.337
2,-CLKREQ_PCIE3,VIA,599.337,TOP,0.000
3,-CLKREQ_PCIE3,VIA,768.006,L8,168.669
4,-CLKREQ_PCIE3,VIA,768.006,TOP,0.000
5,-CLKREQ_PCIE3,VIA(T),1859.664,BOTTOM,1091.658
6,-CLKREQ_PCIE3,R7203.1,1896.078,TOP,36.414
7,-CLKREQ_PCIE3,U7201.30,1936.255,BOTTOM,40.177
8,-CLKREQ_PCIE4,R7001.1,0.000,TOP,0.000
9,-CLKREQ_PCIE4,VIA,22.902,TOP,22.902


In [6]:
for i in netNameList:
    indexList = dfSQSR[dfSQSR["net_name"] == f"{i}"].index.tolist()
    print(indexList)
    
#     dfSQSR.loc[indexList[0],"layer"] = dfSQSR.loc[indexList[1],"layer"]

[0, 1, 2, 3, 4, 5, 6, 7]
[8, 9, 10, 11, 12]
[13, 14, 15, 16, 17, 18]
[19, 20]
[21, 22, 23, 24, 25]
[26, 27, 28, 29, 30, 31, 32, 33, 34]
[35, 36, 37, 38, 39, 40]
[41, 42]
[43, 44, 45, 46, 47, 48]
[49, 50]
[51, 52, 53, 54, 55, 56]
[57, 58]
[59, 60, 61, 62, 63, 64]
[65, 66]
[67, 68, 69, 70, 71, 72]
[73, 74]
[75, 76, 77, 78, 79]
[80, 81, 82, 83, 84]
[85, 86, 87, 88, 89]
[90, 91, 92, 93, 94, 95]
[96]
[97, 98, 99, 100, 101, 102]
[103]
[104, 105, 106, 107, 108]
[109, 110, 111, 112, 113]
[114, 115, 116, 117, 118, 119]
[120]
[121, 122, 123, 124, 125, 126]
[127]
[128, 129, 130, 131, 132]
[133, 134, 135, 136, 137]
[138, 139, 140, 141, 142, 143]
[144]
[145, 146, 147, 148, 149, 150]
[151]
[152, 153, 154, 155, 156]
[157, 158, 159, 160, 161]
[162, 163, 164, 165, 166, 167]
[168]
[169, 170, 171, 172, 173, 174]
[175]
[176, 177, 178]
[179, 180, 181]
[182, 183]
[184, 185]


In [11]:
df = pd.DataFrame(dfSQS)
df1 = pd.DataFrame(dfSQS)
df2 = pd.DataFrame(dfSQS)

In [81]:
dfFinal

,SQS,length
0,-CLKREQ_PCIE0,
1,U58.CF32:J22.52,5492.94
2,U58.CF32:R9562.1,5454.03
3,R9562.1:J22.52,38.908
5,-CLKREQ_PCIE7,
6,U58.CE31:U13.48,7817.27
7,U58.CE31:R29.1,7756.34
8,R29.1:U13.48,60.934
10,-CLKREQ_PCIE8,
11,U58.CE30:J1.52,8661.95


In [28]:
df1 = dfSQSR
df2 = dfSQSR
dfsummarytest = dfsummary
df1.shape

(518, 5)

3

In [49]:
branchPathNum(dfsummarytest)

3

In [71]:

#轉成final SQS資料
column1 = []
column2 = []
for row in range(dfsummarytest.shape[0]):
    column1.append(dfsummarytest.loc[row,"net_name"])
    column1.append(dfsummarytest.loc[row,"start_end"])
    column2.append("")
    column2.append(dfsummarytest.loc[row,"total_length"])
#     print(dfsummarytest.loc[row,"net_name"])
#     print(dfsummarytest.loc[row,"start_end"])
    for num in range(branchPathNum(dfsummarytest)):
#         print(dfsummarytest.loc[row,f"path{num + 1}"])
        column1.append(dfsummarytest.loc[row,f"path{num + 1}"])
        column2.append(dfsummarytest.loc[row,f"length{num + 1}"])
final = {
    "SQS"    : column1,
    "length" :column2
}
#轉成df 順便去掉空值
dfFinal = pd.DataFrame(final).dropna(axis=0)

,SQS,length
0,-CLKREQ_PCIE0,
1,U58.CF32:J22.52,5492.94
2,U58.CF32:R9562.1,5454.03
3,R9562.1:J22.52,38.908
5,-CLKREQ_PCIE7,
6,U58.CE31:U13.48,7817.27
7,U58.CE31:R29.1,7756.34
8,R29.1:U13.48,60.934
10,-CLKREQ_PCIE8,
11,U58.CE30:J1.52,8661.95


In [ ]:

df1 = df1.fillna("na")
df1


summary1 = []
for row in range(dfsummarytest.shape[0]):
    for element in df1.loc[row]:
#         print(element)
#         print(type(element))
        if element != "na":
            summary1.append(element)
            
summary1


summary2 = []
for row in range(df1.shape[0]):
    summary2 += [ele for ele in df1.loc[row]]

In [18]:
for i in netNameList:
    indexList = df1[df1["net_name"] == f"{i}"].index.tolist()
    path = ""
    connIdxList = []
    
    for idx in indexList:
        if not re.findall("VIA", df1.loc[idx, "location"]): #找出connnector index 排出VIA, VIA(T)
            connIdxList.append(idx)
    print("net name:", i)
    print("index list:", indexList)
    print("connector's index:", connIdxList)
    length = len(connIdxList)
    
    #找出connector 在每條indexlist 裡面的index
    num = 0
    netIndex = dfsummarytest[dfsummarytest["net_name"] == f"{i}"].index.tolist()
    
    #把起始點&終點 加入summary
    start_end = f"{df1.loc[indexList[0], 'location']}:{df1.loc[indexList[-1], 'location']}"
    print(start_end)
    dfsummarytest.loc[netIndex, "start_end"] = start_end
    for idx in range(length - 1):
        num +=1 
        pathIdx = []
        pathIdx.append(indexList.index(connIdxList[idx]))
        pathIdx.append(indexList.index(connIdxList[idx + 1]))
        
        print("connector's Idx in IndexList :", pathIdx)
        
        #抓出兩兩conn
        connector = ""
        for j in pathIdx:
            connector += df1.loc[indexList[j], "location"] + ":"
        connector = connector[:-1] 
        print("conn path:", connector)
        
        dfsummarytest.loc[netIndex, f"path{num}"] = connector
        
        #算出兩兩conn間的長度
        branchLen = 0
        for k in indexList[ pathIdx[0]+1 : pathIdx[1]+1]:
            print("index:", k)
            print("gap",df1.loc[k, "gap"])
            branchLen += df1.loc[k, "gap"]
        print("sub length:", branchLen)
        dfsummarytest.loc[netIndex, f"length{num}"] = branchLen
        
        print("-"*100)


net name:  -CLKREQ_PCIE0
index list: [0, 3, 4, 5, 6]
connector's index: [0, 5, 6]
U58.CF32:J22.52
connector's Idx in IndexList : [0, 3]
conn path: U58.CF32:R9562.1
index: 3
gap 5060.579
index: 4
gap 365.58700000000044
index: 5
gap 27.868999999999687
sub length: 5454.035
----------------------------------------------------------------------------------------------------
connector's Idx in IndexList : [3, 4]
conn path: R9562.1:J22.52
index: 6
gap 38.90800000000036
sub length: 38.90800000000036
----------------------------------------------------------------------------------------------------
net name:  -CLKREQ_PCIE7
index list: [7, 10, 11, 12]
connector's index: [7, 11, 12]
U58.CE31:U13.48
connector's Idx in IndexList : [0, 2]
conn path: U58.CE31:R29.1
index: 10
gap 7536.716
index: 11
gap 219.6199999999999
sub length: 7756.336
----------------------------------------------------------------------------------------------------
connector's Idx in IndexList : [2, 3]
conn path: R29.1:U13.48

index: 105
gap 38.112
index: 106
gap 1579.336
index: 107
gap 337.8309999999999
index: 108
gap 2171.7909999999997
index: 109
gap 58.91700000000037
sub length: 4185.987
----------------------------------------------------------------------------------------------------
net name:  PCIE0_RXN
index list: [110, 111, 113, 115]
connector's index: [110, 115]
U58.CB5:J22.41
connector's Idx in IndexList : [0, 3]
conn path: U58.CB5:J22.41
index: 111
gap 20.792
index: 113
gap 4217.237
index: 115
gap 1.6109999999998763
sub length: 4239.64
----------------------------------------------------------------------------------------------------
net name:  PCIE0_RXP
index list: [116, 117, 119, 121]
connector's index: [116, 121]
U58.CB6:J22.43
connector's Idx in IndexList : [0, 3]
conn path: U58.CB6:J22.43
index: 117
gap 20.521
index: 119
gap 4214.369000000001
index: 121
gap 1.7269999999998618
sub length: 4236.617
-----------------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------
net name:  PCIE8_L1_TXN
index list: [355, 358, 360]
connector's index: [355, 360]
U58.BN4:C8668.1
connector's Idx in IndexList : [0, 2]
conn path: U58.BN4:C8668.1
index: 358
gap 6542.253
index: 360
gap 0.0
sub length: 6542.253
----------------------------------------------------------------------------------------------------
net name:  PCIE8_L1_TXN_CONN
index list: [361, 362]
connector's index: [361, 362]
C8668.2:J1.35
connector's Idx in IndexList : [0, 1]
conn path: C8668.2:J1.35
index: 362
gap 66.111
sub length: 66.111
----------------------------------------------------------------------------------------------------
net name:  PCIE8_L1_TXP
index list: [363, 366, 368]
connector's index: [363, 368]
U58.BN3:C8669.1
connector's Idx in IndexList : [0, 2]
conn path: U58.BN3:C8669.1
index: 366
gap 6542.923
index: 368
gap 0.0
sub length: 6542.923
-------------------------------------------

index: 486
gap 381.886
sub length: 381.886
----------------------------------------------------------------------------------------------------
net name:  USB3P4_RXP
index list: [487, 489, 490, 492]
connector's index: [487, 492]
U58.BW5:J5.3
connector's Idx in IndexList : [0, 3]
conn path: U58.BW5:J5.3
index: 489
gap 16.109
index: 490
gap 7375.023999999999
index: 492
gap 0.7790000000004511
sub length: 7391.912
----------------------------------------------------------------------------------------------------
net name:  USB3P4_RXP_CONN
index list: [493, 494]
connector's index: [493, 494]
J5.4:J33.6
connector's Idx in IndexList : [0, 1]
conn path: J5.4:J33.6
index: 494
gap 378.147
sub length: 378.147
----------------------------------------------------------------------------------------------------
net name:  USB3P4_TXN
index list: [495, 496, 497, 498, 500]
connector's index: [495, 500]
U58.BW2:C298.2
connector's Idx in IndexList : [0, 4]
conn path: U58.BW2:C298.2
index: 496
gap 1.895


index: 636
gap 4273.416
index: 637
gap 79.3760000000002
sub length: 4352.792
----------------------------------------------------------------------------------------------------
net name:  USBP7-_CONN
index list: [638, 639, 640]
connector's index: [638, 640]
R793.2:J22.9
connector's Idx in IndexList : [0, 2]
conn path: R793.2:J22.9
index: 639
gap 71.164
index: 640
gap 61.693
sub length: 132.857
----------------------------------------------------------------------------------------------------
net name:  USBP8+
index list: [641, 643, 644, 645, 647]
connector's index: [641, 647]
U58.CB9:R3137.1
connector's Idx in IndexList : [0, 4]
conn path: U58.CB9:R3137.1
index: 643
gap 16.227
index: 644
gap 762.5930000000001
index: 645
gap 2233.6589999999997
index: 647
gap 0.0
sub length: 3012.479
----------------------------------------------------------------------------------------------------
net name:  USBP8+_CONN
index list: [648, 649]
connector's index: [648, 649]
J11.12:R3137.2
connector's I

In [19]:
dfsummarytest

,net_name,total_length,path1,length1,path2,length2,path3,length3,start_end
0,-CLKREQ_PCIE0,5492.943,U58.CF32:R9562.1,5454.035,R9562.1:J22.52,38.908,NaN,NaN,U58.CF32:J22.52
1,-CLKREQ_PCIE7,7817.270,U58.CE31:R29.1,7756.336,R29.1:U13.48,60.934,NaN,NaN,U58.CE31:U13.48
2,-CLKREQ_PCIE8,8661.951,U58.CE30:R2038.1,8599.129,R2038.1:J1.52,62.822,NaN,NaN,U58.CE30:J1.52
3,-CLKREQ_PCIE12,9541.714,U182.AC5:R2908.1,177.307,R2908.1:U58.CF31,9364.407,NaN,NaN,U182.AC5:U58.CF31
4,-PCIE0_CLK_100M,4178.524,U58.AW2:J22.53,4178.524,NaN,NaN,NaN,NaN,U58.AW2:J22.53
...,...,...,...,...,...,...,...,...,...
138,USBP9-_CONN,92.429,J16.A1:R799.2,92.429,NaN,NaN,NaN,NaN,J16.A1:R799.2
139,USB_ON1,3639.200,U23.A2:U53.4,3639.200,NaN,NaN,NaN,NaN,U23.A2:U53.4
140,USB_ON2,6162.597,U23.A1:J65.18,6162.597,NaN,NaN,NaN,NaN,U23.A1:J65.18
141,VCC0R9_TBT_USB,78.055,U182.R15:U182.R16,36.416,U182.R16:C2980.2,19.139,C2980.2:C2981.2,22.5,U182.R15:C2981.2


In [ ]:
for i in netNameList:
    indexList = dfSQSR[dfSQSR["net_name"] == f"{i}"].index.tolist()
    path = ""
    connIdx = [] #每一條net 的conn list
    
    for idx in indexList:
        
        if not re.findall("VIA", dfSQSR.loc[idx, "location"]): #找出connnector index 排出VIA, VIA(T)
            connIdx.append(idx)

    length = len(connIdx)
    
    for j in range(length - 1):
        path = dfSQSR.loc[connIdx[j], "location"] + ":" + dfSQSR.loc[connIdx[j + 1], "location"]
        index = dfsummary[dfsummary["net_name"] == f"{i}"].index.tolist()
        dfsummary.loc[index, f"path{j}"] = path

In [371]:
df1[df1["net_name"] == " PCIE12_L2_TXP_C"]
a = df1[df1["net_name"] == " PCIE12_L2_TXP_C"].index.tolist()

[322, 323]

In [382]:
df1.loc[322]["length"]
df1.loc[323]["length"]

260.186

In [409]:
length = len(df1[df1["net_name"] == " PCIE12_L2_TXP_C"].index.tolist())
for index in df1[df1["net_name"] == " PCIE12_L2_TXP_C"].index.tolist()[::-1]:

    if length > 1:
        gap = df1.loc[index]["length"] - df1.loc[index-1]["length"]
    else:
        gap = 0
    
    length -= 1
    print(index, gap)

323 260.186
322 0


In [398]:
df1.loc[323]["length"]
df1.loc[322]["length"]

0.0

In [383]:
df1.loc[322:323]["length"]

322      0.000
323    260.186
Name: length, dtype: float64

In [384]:
df1.loc[322:323]["length"].tolist()

[0.0, 260.186]

In [335]:
df1.loc["net_name"]= "PCIE12_L2_TXP_C"